# 实验九 太阳能电池的基本特性研究

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
import random
import math
import numpy as np
import pandas as pd
import os

In [ ]:
# 如图表中字体无法显示，在此修改字体
plt.rcParams['font.size'] = 9
plt.rcParams['font.sans-serif'] = ['Source Han Sans CN']

In [ ]:
%config InlineBackend.figure_format = 'retina'

In [ ]:
colors = list(mpl.colors.XKCD_COLORS.keys())
random.shuffle(colors)
plt.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)

print('colors =', colors[:5])
plt.figure(figsize=(5,1))
plt.title("Does these colors look pretty to you? If not, run this cell again.")
plt.gca().axis('off')
plt.scatter(range(5), [1]*5, color=colors[:5], s=512)
plt.show()

In [ ]:
plt.rcParams['axes.prop_cycle'] = mpl.cycler(color=colors)

### 1. 硅太阳能电池的暗特性测量

In [ ]:
st1 = pd.read_excel("./data-raw.xlsx", sheet_name="st1-太阳能电池的暗伏安特性测量", header=[0,1])


In [ ]:
g1 = st1.sort_values(by=st1.columns[0])
U = g1.iloc[:,0]
I = g1.iloc[:,1]

In [ ]:
os.makedirs("./fig", exist_ok=True)

In [ ]:
plt.figure(figsize=(3, 3))
plt.plot(U, I, '.-')
plt.xlabel(r'$U \: \rm (V)$')
plt.ylabel(r'$I \: \rm (mA)$')
plt.grid()
# plt.title("单晶硅的暗伏安特性曲线")
plt.savefig("./fig/st1-g1.svg", bbox_inches="tight")
plt.show()

In [ ]:
g2 = st1.iloc[:, 2:4]
g2.iloc[:,0] = g2.iloc[:,0].fillna(st1.iloc[:,0])
g2 = g2.dropna().sort_values(by=g2.columns[0])

In [ ]:
U = g2.iloc[:,0]
I = g2.iloc[:,1]
plt.figure(figsize=(3, 3))
plt.plot(U, I, '.-', color=colors[1])
plt.xlabel(r'$U \: \rm (V)$')
plt.ylabel(r'$I \: \rm (mA)$')
# plt.title("非晶硅的暗伏安特性曲线")
plt.grid()
plt.savefig("./fig/st1-g2.svg", bbox_inches="tight")
plt.show()

### 2. 开路电压、短路电流与光强关系测量

In [ ]:
st2 = pd.read_excel("./data-raw.xlsx", sheet_name="st2-开路电压、短路电流与光强关系测量", index_col=[0,1])

In [ ]:
I = st2.iloc[0]

In [ ]:
plt.figure(figsize=(6, 4))
plt.plot(st2.iloc[0], st2.iloc[1], '.-', label="单晶硅")
plt.plot(st2.iloc[0], st2.iloc[3], '.-', label="非晶硅")
plt.ylabel(r'$V_{\rm OC} \: \rm (V)$')
plt.xlabel(r'$I \: \rm (W/m^2)$')
# plt.title("两种太阳能电池的开路电压随光强变化的关系")
plt.xlim(0, 900)
plt.grid()
plt.legend()
plt.savefig("./fig/st2-voc.svg", bbox_inches="tight")
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(st2.iloc[0], st2.iloc[2], '.-', label="单晶硅")
plt.ylabel(r'$I_{\rm SC} \: \rm (mA)$')
plt.xlabel(r'$I \: \rm (W/m^2)$')
# plt.title("单晶硅太阳能电池的短路电流随光强变化的关系")
plt.xlim(0, 900)
plt.ylim(0, 100)
plt.grid()
# plt.legend()
plt.savefig("./fig/st2-isc-monocryst.svg", bbox_inches="tight")
plt.show()

In [ ]:
plt.figure(figsize=(4, 3))
plt.plot(st2.iloc[0], st2.iloc[4], '.-', color=colors[1], label="非晶硅")
plt.ylabel(r'$I_{\rm SC} \: \rm (mA)$')
plt.xlabel(r'$I \: \rm (W/m^2)$')
# plt.title("非晶硅太阳能电池的短路电流随光强变化的关系")
plt.xlim(0, 900)
plt.ylim(0, 8)
plt.grid()
# plt.legend()
plt.savefig("./fig/st2-isc-amorph.svg", bbox_inches="tight")
plt.show()

### 3. 太阳能电池输出特性测试

In [ ]:
st3 = pd.read_excel("./data-raw.xlsx", sheet_name="st3-太阳能电池输出特性测试", index_col=[0,1], dtype=str)

In [ ]:
# 太阳能电池面积
S = 50e-3*50e-3 # (m^2)
# 光强
I_light = float(st3.columns[0]) # (W/m^2)

In [ ]:
def calc_st3(V, I, name, outfile):
    # V = st3.iloc[0] # (V)
    # I = st3.iloc[1] # (mA)
    P = V*I # (mW)
    R = V/I*1000 # (Ω)
    
    fig, ax1 = plt.subplots()
    
    fig.set_size_inches(5, 3)
    ax1.set_xlabel('$V$ (V)')
    ax1.set_ylabel('$I$ (mA)')
    ax1.plot(V, I, '.-', color=colors[2], label="输出电流 $I$")
    
    ax2 = ax1.twinx()
    ax2.set_ylabel('$P_o$ (mW)')
    ax2.plot(V, P, '+-', color=colors[3], label="输出功率 $P_o$")

    # ax1.legend(loc='lower center')
    # ax2.legend(loc='center left')
    fig.legend(loc=(0.4, 0.2))
    # plt.title(name + "的伏安特性及功率曲线")
    plt.savefig(outfile, bbox_inches="tight", dpi=600)
    plt.show()

    print(f"=== {name} ===")
    print("最大功率：")
    print(f"{P.max()} (mW)")
    print("最佳匹配负载电阻：")
    print(f"{R[P.idxmax()]} (Ω)")

    Isc = I.iloc[0] # (mA)
    Voc = V.max() # (V)
    print()
    print("短路电流：")
    print(f"Isc = {Isc} (mA)")
    print("开路电压：")
    print(f"Voc = {Voc} (V)")
    print("填充因子：")
    print(f" FF = {P.max()/(Isc*Voc)}")
    print("转换效率：")
    print(f" ηₛ = {P.max()*1e-3/(S*I_light)*100} %")

    def _format_power(x):
        return '' if np.isnan(x) else "{:.2f}".format(x)

    return P.apply(_format_power)
    
    

In [ ]:
st3.iloc[2] = calc_st3(st3.iloc[0].astype(float), st3.iloc[1].astype(float), "单晶硅", "./fig/st3-monocryst.svg")

In [ ]:
st3.iloc[5] = calc_st3(st3.iloc[3].astype(float), st3.iloc[4].astype(float), "非晶硅", "./fig/st3-amorph.svg")

In [ ]:
import shutil

filename_out = "./data.xlsx"
shutil.copyfile("./data-raw.xlsx", filename_out)

# https://stackoverflow.com/questions/20219254/how-to-write-to-an-existing-excel-file-without-overwriting-data-using-pandas
with pd.ExcelWriter(filename_out, mode="a", engine='openpyxl', if_sheet_exists="overlay") as writer:
    st3.to_excel(writer, sheet_name="st3-太阳能电池输出特性测试"#, index=None, startcol=2, startrow=3
                )  # Start at C4 

print(f'输出功率计算后已写入文件 {filename_out}。')
